In [1]:
import sys
sys.path.append('../code/')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
import matplotlib.pyplot as plt
from sklearn.model_selection import ShuffleSplit
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from nilearn import datasets, plotting

from mamba_ssm import Mamba

import utils
from train_ann_sweep import get_generators
import pickle
from train_ann_sweep import get_generators

device = torch.device("cuda:0")
# device = torch.device("cpu")

In [3]:
atlas = datasets.fetch_atlas_msdl()
# Loading atlas image stored in 'maps'
atlas_filename = atlas["maps"]
# Loading atlas data stored in 'labels'
labels = atlas["labels"]

df = pd.read_pickle('../data/developmental_df.pkl')
n_subjects = len(np.unique(df['subj'].values))

time_series = df.iloc[:,:-1].values

# pca = PCA(n_components=39)
# points = pca.fit_transform(time_series)


In [4]:
with open('../data/ann_bottleneck_sweep_results.pkl', 'rb') as f:
    metadata = pickle.load(f)
print(metadata['bottleneck_values'])

cv_dict = metadata['bottleneck_1']['fold_0']['metadata']['cv_dict']

[1, 3, 5, 7, 9, 11, 13, 15]


In [30]:
def get_autoencoder_mse(training_set, validation_set, fold=0, bottleneck=15):
    fold
    input_size = 39
    encoder_layer_size = [500, 300, 100]
    decoder_layer_size = encoder_layer_size[::-1]


    model = utils.model_ann_autoencoder(input_size, encoder_layer_size, decoder_layer_size, bottleneck=bottleneck).to(device)
    state_dict = torch.load(f'../models/ann/bottleneck/ann_fold{fold}_bottleneck{bottleneck}.pt')
    model.load_state_dict(state_dict)

    model.eval()
    with torch.no_grad():
        y_pred = model(training_set[:].float().to(device))[0].detach().cpu().numpy()
    return y_pred

   

def get_pca_mse(training_set, validation_set, fold=0, bottleneck=15):

    explained_var = np.cumsum(pca.explained_variance_ratio_)
    explained_var = np.concatenate([[0], explained_var])
    thresh_pos = np.argmin(np.abs(explained_var - 0.8))




In [31]:
fold = 0
bottleneck = 15

generators, data_arrays = get_generators(df, cv_dict, fold=fold)
training_generator, _, validation_generator, _ = generators
training_set, _, validation_set = data_arrays

In [32]:
get_autoencoder_mse(training_set, validation_set, fold=0, bottleneck=15)

array([[[ 0.31770268,  0.36189893,  1.4210453 , ...,  0.10869498,
          0.49367625,  0.65483284],
        [-0.32437745,  0.28659242,  1.9209124 , ...,  1.4178104 ,
         -1.2053497 ,  1.5954993 ],
        [ 0.43880537,  2.5116465 ,  2.6639009 , ...,  1.9699343 ,
         -1.0985781 ,  2.7051704 ],
        ...,
        [ 1.0592061 ,  1.9051914 , -0.10697276, ..., -0.35990676,
         -0.02265709, -0.15284565],
        [-0.8116818 ,  0.9465032 ,  0.05225476, ...,  1.714603  ,
          0.7222819 ,  0.7595777 ],
        [-1.258226  , -0.4820231 , -0.07289536, ...,  1.0465602 ,
         -1.0027535 ,  2.5083678 ]],

       [[-0.32437745,  0.28659242,  1.9209124 , ...,  1.4178104 ,
         -1.2053497 ,  1.5954993 ],
        [ 0.43880537,  2.5116465 ,  2.6639009 , ...,  1.9699343 ,
         -1.0985781 ,  2.7051704 ],
        [ 0.7723978 ,  0.58849937,  2.1524105 , ...,  0.42968118,
          0.92690057,  0.08305342],
        ...,
        [-0.8116818 ,  0.9465032 ,  0.05225476, ...,  